In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Set display options to show all columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Create a path to where the data is stored.
path = 'C:/Users/kaan/ML_2'

In [4]:
X = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_cleaned.csv'))

In [5]:
y = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'predictions_cleaned.csv'))

In [6]:
X.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,STOCKHOLM_humidity,MUNCHENB_pressure
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,8,0.98,1.0304
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,6,0.62,1.0292
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.0

In [7]:
X.shape

(22950, 135)

In [8]:
y.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
y.shape

(22950, 15)

In [10]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)

In [13]:
X = X.reshape(-1,15,9)

In [14]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

# Keras Model

In [15]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh')) # Don't use relu here!

C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,423 (36.81 KB)

 Trainable params: 9,423 (36.81 KB)

 Non-trainable params: 0 (0.00 B)

# Compile and run the model

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 4s - 4ms/step - accuracy: 0.0834 - loss: 24.3140
Epoch 2/30
1076/1076 - 3s - 3ms/step - accuracy: 0.1574 - loss: 24.5105
Epoch 3/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1490 - loss: 24.5983
Epoch 4/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1035 - loss: 24.9126
Epoch 5/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0716 - loss: 24.8786
Epoch 6/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0508 - loss: 24.3879
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0448 - loss: 24.3886
Epoch 8/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0521 - loss: 24.4290
Epoch 9/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0368 - loss: 24.5117
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0290 - loss: 24.4643
Epoch 11/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0271 - loss: 24.1413
Epoch 12/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0458 - loss: 24.4209
Epoch 13/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0435 - loss: 24.6562
Epoch 14/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0305 - l

# Confusion matrix

In [19]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [20]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [21]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   
Pred        DEBILT  HEATHROW  KASSEL  MAASTRICHT  STOCKHOLM  VALENTIA
True                                                                 
BASEL            4         9      20        3630          5        14
BELGRADE         0         0       0        1092          0         0
BUDAPEST         0         0       0         214          0         0
DEBILT           0         0       0          82          0         0
DUSSELDORF       0         0       0          29          0         0
HEATHROW         0         0       0          82          0         0
KASSEL           0         0       0          11          0         0
LJUBLJANA        0         0       0          61          0         0
MAASTRICHT       0         0       0           9          0         0
MADRID           0         0       0         458          0         0
MUNCHENB         0         0       0           8          0         0
OSLO             0         0       0          

In [22]:
model.evaluate(X_test, y_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0016 - loss: 27.6323       


[27.632286071777344, 0.0015684907557442784]

# Model improvement steps

In [23]:
epochs = 30
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
538/538 - 2s - 3ms/step - accuracy: 0.0332 - loss: 20.1079
Epoch 2/30
538/538 - 1s - 2ms/step - accuracy: 0.0340 - loss: 19.1850
Epoch 3/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1906
Epoch 4/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1906
Epoch 5/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1924
Epoch 6/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1943
Epoch 7/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1981
Epoch 8/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1999
Epoch 9/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1999
Epoch 10/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.1990
Epoch 11/30
538/538 - 1s - 2ms/step - accuracy: 0.0342 - loss: 19.2018
Epoch 12/30
538/538 - 1s - 2ms/step - accuracy: 0.0345 - loss: 19.1971
Epoch 13/30
538/538 - 1s - 2ms/step - accuracy: 0.0349 - loss: 19.1924
Epoch 14/30
538/538 - 1s - 2ms/step - accuracy: 0.0349 - loss: 19.1924
Epoch 15/30
538

In [26]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [27]:
print(confusion_matrix(y_test, model.predict(X_test)))
model.evaluate(X_test, y_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BUDAPEST  LJUBLJANA  MUNCHENB  SONNBLICK  VALENTIA
True                                                          
BASEL           3047          1         1          1       632
BELGRADE        1090          0         0          2         0
BUDAPEST         214          0         0          0         0
DEBILT            82          0         0          0         0
DUSSELDORF        29          0         0          0         0
HEATHROW          82          0         0          0         0
KASSEL            11          0         0          0         0
LJUBLJANA         61          0         0          0         0
MAASTRICHT         9          0         0          0         0
MADRID           430          1         0          1        26
MUNCHENB           8          0         0          0         0
OSLO               5          0         0          0         0
STOCKHOLM          4          0         0          0         0
VALENTIA    

[18.634965896606445, 0.037295226007699966]

In [28]:
# 3rd test 
epochs = 50
batch_size = 32
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

C:\Users\kaan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/50
538/538 - 2s - 3ms/step - accuracy: 0.3105 - loss: 25.6823
Epoch 2/50
538/538 - 1s - 2ms/step - accuracy: 0.3568 - loss: 26.0566
Epoch 3/50
538/538 - 1s - 2ms/step - accuracy: 0.3907 - loss: 24.6576
Epoch 4/50
538/538 - 1s - 2ms/step - accuracy: 0.4575 - loss: 22.8483
Epoch 5/50
538/538 - 1s - 2ms/step - accuracy: 0.4642 - loss: 22.9008
Epoch 6/50
538/538 - 1s - 2ms/step - accuracy: 0.4010 - loss: 26.2270
Epoch 7/50
538/538 - 1s - 2ms/step - accuracy: 0.3780 - loss: 26.2739
Epoch 8/50
538/538 - 1s - 2ms/step - accuracy: 0.3700 - loss: 26.2701
Epoch 9/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 10/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 11/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 12/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 13/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 14/50
538/538 - 1s - 2ms/step - accuracy: 0.3692 - loss: 26.2701
Epoch 15/50
538

In [30]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

print(confusion_matrix(y_test, model.predict(X_test)))
model.evaluate(X_test, y_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  MAASTRICHT  MADRID  SONNBLICK  STOCKHOLM
True                                                                 
BASEL        1807       948          12     717        197          1
BELGRADE      748       282           1       3         58          0
BUDAPEST      142        64           0       2          6          0
DEBILT         74         8           0       0          0          0
DUSSELDORF     24         5           0       0          0          0
HEATHROW       40        39           0       3          0          0
KASSEL          8         3           0       0          0          0
LJUBLJANA      26        29           1       1          4          0
MAASTRICHT      1         7           0       0          1          0
MADRID        143       289           1      18          7          0
MUNCHENB        0         8           0       0          0          0
OSLO            4         1           0       0

[26.042327880859375, 0.3672011196613312]